In [1]:
install.packages("vars")

Installing package into 'C:/Users/Natty/Documents/R/win-library/3.6'
(as 'lib' is unspecified)

also installing the dependencies 'zoo', 'strucchange', 'urca', 'lmtest', 'sandwich'




package 'zoo' successfully unpacked and MD5 sums checked
package 'strucchange' successfully unpacked and MD5 sums checked
package 'urca' successfully unpacked and MD5 sums checked
package 'lmtest' successfully unpacked and MD5 sums checked
package 'sandwich' successfully unpacked and MD5 sums checked
package 'vars' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\Natty\AppData\Local\Temp\RtmpoHWMzD\downloaded_packages


In [ ]:
library("vars")